# 경사 하강법을 이용한 얕은 신경망 학습


In [33]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [34]:
EPOCHS=1000

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [35]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.d1=tf.keras.layers.Dense(128,input_dim=2,activation='sigmoid')
        self.d2=tf.keras.layers.Dense(10,activation='softmax')
    def call(self,x,training=None,mask=None):
        x=self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [36]:
@tf.function
def train_step(model,inputs,labels,loss_object,optimizer,train_loss,train_metric):
    with tf.GradientTape() as tape:
        predictions=model(inputs)
        loss=loss_object(labels,predictions)
    gradients=tape.gradient(loss,model.trainable_variables) #grad(loss) 이자 df/dx
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    train_loss(loss)
    train_metric(labels,predictions)

## 데이터셋 생성, 전처리

In [37]:
np.random.seed(0)# 랜덤값동일하게 만들 수 잇음 
pts=list()#input값, 2가지 스칼라가 들어온다. 길이가 2인게 들어간다. 
labels=list()#10개의 클래스 중 하나를 표현한다. 10개의 클래스를 가지는 포인터를 생성할거다. 
center_pts=np.random.uniform(-8.0,8.0,(10,2))
for label,center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt+np.random.randn(*center_pt.shape))
        labels.append(label)
        
pts=np.stack(pts,axis=0).astype(np.float32)
labels=np.stack(labels,axis=0)
train_ds=tf.data.Dataset.from_tensor_slices((pts,labels)).shuffle(1000).batch(32)

## 모델 생성

In [38]:
model=MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [39]:
loss_object=tf.keras.losses.SparseCategoricalCrossentropy() #몇번째가 1이냐를 0123으로 바꿔주는거임
optimizer=tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [40]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_accuracy=tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프

In [41]:
for epoch in range(EPOCHS):
    for x,label in train_ds:
        train_step(model,x,label,loss_object, optimizer,train_loss,train_accuracy)
        
    template='Epoch{}, Loss:{}, Accuracy:{}'
    print(template.format(epoch+1,train_loss.result(),train_accuracy.result()*100))

Epoch1, Loss:2.188542127609253, Accuracy:22.799999237060547
Epoch2, Loss:2.0076956748962402, Accuracy:35.25
Epoch3, Loss:1.8729147911071777, Accuracy:44.233333587646484
Epoch4, Loss:1.7646178007125854, Accuracy:48.599998474121094
Epoch5, Loss:1.6756665706634521, Accuracy:53.2400016784668
Epoch6, Loss:1.598551630973816, Accuracy:55.966670989990234
Epoch7, Loss:1.532668113708496, Accuracy:58.499996185302734
Epoch8, Loss:1.4730664491653442, Accuracy:61.375
Epoch9, Loss:1.4184874296188354, Accuracy:63.355552673339844
Epoch10, Loss:1.369571566581726, Accuracy:65.24000549316406
Epoch11, Loss:1.3254328966140747, Accuracy:66.76363372802734
Epoch12, Loss:1.2849115133285522, Accuracy:68.28333282470703
Epoch13, Loss:1.2483317852020264, Accuracy:69.43846130371094
Epoch14, Loss:1.2143980264663696, Accuracy:70.55000305175781
Epoch15, Loss:1.1821190118789673, Accuracy:71.64666748046875
Epoch16, Loss:1.152413010597229, Accuracy:72.5875015258789
Epoch17, Loss:1.12441885471344, Accuracy:73.4117660522461

## 데이터셋 및 학습 파라미터 저장

In [49]:
import numpy as np
np.savez_compressed('ch2_dataset.npz , inputs=pts, labels=labels')
W_h,b_h=model.d1.get_weights()
W_o,b_o=model.d2.get_weights()
W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameter.npz',
                   W_h=W_h,
                   b_h=b_h,
                   W_o=W_o,
                  b_o=b_o)